In [1]:
import pandas as pd
import numpy as np
%matplotlib inline

In [2]:
data = pd.read_csv('data/data_update-2017-10.csv')

In [3]:
import unicodedata

def elimina_tildes(s):
    return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

def descripcion_token(x):
    x = elimina_tildes(x)
    words = ['parrilla','suite','cochera', 'placard', 'pileta', 'terraza',\
             'vestidor', 'patio', 'hall', 'servicio', 'muebles', 'balcon', 'jard','aire acondicionado', \
             'quincho', 'fondo', 'piscina', 'garage', 'parque', 'estrenar', \
             'galer', 'dependencia', 'buena','calefaccion','radiante', 'seguridad', \
             'chalet','gimnasio','vista']
    result = ''
    for word in words:
        if word in x:
            result += '|' + word
    return result
def place_name_from_parent(x): ## Segun vi el 3 item del place es generalmente el lugar
    list = x.split('|')
    if(len(list) < 5):
        return np.nan
    else:
        return list[3]
    
def rescatar_barrios_no_oficiales(x):
    if x == 'Once':
        return ("Balvanera")
    elif x == 'Bajo Belgrano':
        return ("Belgrano")
    elif x == 'Alto Palermo':
        return ("Palermo")
    elif x == 'Tribunales':
        return ("San Nicolás")
    elif x == 'Las Cañitas':
        return("Palermo")
    elif x == 'Capital Federal':
        return(np.nan)
    else:
        return x
    
def recuperar_cant_amb(x):
    "Recupera el dato amb en x (puede ser la descripcion o el titulo)"
    try:
        pos = 0
        x = x.lower()
        if "ambientes" in x:
            pos = x.rfind("ambientes")
        elif "amb" in x:
            pos = x.rfind("amb")
        if pos != 0:
            try:
                return (np.float64(x[pos -2]))
            except ValueError:
                if ("dos amb" in x) or ("dos ambientes" in x):
                    return 2.0
                elif ("tres amb" in x) or ("tres ambientes" in x):
                    return 3.0
                elif ("cuatro amb" in x) or ("cuatro ambientes" in x):
                    return 4.0
                elif ("cinco amb" in x) or ("cinco ambientes" in x):
                    return 5.0
                elif ("seis amb" in x) or ("seis ambientes" in x):
                    return 6.0
                return np.nan
    except AttributeError:
        return None


In [4]:
data.loc[:,'place_name'] = data.place_with_parent_names.apply(place_name_from_parent) ## esto quita muchos problemas 
                                                                                    ## de place name como nombres de country
                                                                                    ## o palermo soho
data.loc[data.description.notnull(),'description'] = data.loc[data.description.notnull(),'description'].apply(descripcion_token)

In [7]:
data.loc[:,'place_name'] = data.place_name.apply(rescatar_barrios_no_oficiales)
data['created_on'] = pd.to_datetime(data['created_on']) ## conversion de tipo
data['month'] = data.created_on.apply(lambda x: x.month)
data['year'] = data.created_on.apply(lambda x: x.year)
data['day'] = data.created_on.apply(lambda x: x.day)

In [8]:
data.dropna(axis=0,subset=['price_aprox_usd'],inplace=True) ## drop de los datos sin precio
data = data.loc[data['price_aprox_usd'] > 0,:]

In [9]:
data.loc[data.rooms.isnull(),'rooms'] = data.description.apply(recuperar_cant_amb)
data.loc[data.rooms.isnull(),'rooms'] = data.title.apply(recuperar_cant_amb)
data.loc[data.surface_total_in_m2.isnull(),'surface_total_in_m2'] = data.surface_covered_in_m2

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284732 entries, 0 to 317037
Data columns (total 26 columns):
id                            284732 non-null object
created_on                    284732 non-null datetime64[ns]
property_type                 284732 non-null object
place_name                    278472 non-null object
place_with_parent_names       284732 non-null object
state_name                    284732 non-null object
geonames_id                   224194 non-null float64
lat-lon                       202676 non-null object
lat                           202676 non-null float64
lon                           202676 non-null float64
price                         284732 non-null float64
currency                      284732 non-null object
price_aprox_local_currency    284732 non-null float64
price_aprox_usd               284732 non-null float64
surface_total_in_m2           266756 non-null float64
surface_covered_in_m2         257298 non-null float64
price_usd_per_m2         

In [12]:
data.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,month,year,day
count,2.241940e+05,202676.000000,202676.000000,2.847320e+05,2.847320e+05,2.847320e+05,2.667560e+05,2.572980e+05,2.012090e+05,2.541550e+05,36551.000000,168778.000000,3.620700e+04,284732.000000,284732.000000,284732.000000
mean,3.498000e+06,-34.557248,-58.525949,3.346441e+05,4.267056e+06,2.545583e+05,7.756810e+03,7.948382e+03,2.001651e+03,4.427643e+03,14.513119,2.931780,3.695044e+03,6.425628,2016.229497,17.467334
std,4.424446e+05,1.376284,1.120437,7.408342e+05,6.123039e+06,3.653688e+05,3.872343e+06,3.942872e+06,4.586174e+03,2.073286e+04,105.542985,1.461706,7.465248e+04,2.818749,0.903856,8.611257
min,3.427208e+06,-51.626650,-122.419415,2.600000e+03,4.757708e+04,2.754180e+03,0.000000e+00,0.000000e+00,8.000000e-02,5.000000e-05,1.000000,0.000000,1.000000e+00,1.000000,2012.000000,1.000000
25%,3.429595e+06,-34.629218,-58.579659,1.050000e+05,1.660724e+06,9.900000e+04,4.800000e+01,4.600000e+01,1.120690e+03,1.527778e+03,2.000000,2.000000,1.000000e+03,4.000000,2016.000000,10.000000
50%,3.430787e+06,-34.592540,-58.481101,1.720000e+05,2.683515e+06,1.600000e+05,8.300000e+01,7.700000e+01,1.785714e+03,2.095532e+03,3.000000,3.000000,1.900000e+03,7.000000,2017.000000,19.000000
75%,3.435719e+06,-34.518473,-58.417314,3.270000e+05,4.808730e+06,2.890000e+05,2.000000e+02,1.600000e+02,2.450000e+03,2.896774e+03,6.000000,4.000000,3.800000e+03,8.000000,2017.000000,24.000000
max,6.693230e+06,52.619706,58.496000,1.234568e+08,7.984250e+08,5.000000e+07,2.000000e+09,2.000000e+09,1.271027e+06,4.000000e+06,7071.000000,40.000000,1.000150e+07,12.000000,2017.000000,31.000000


In [13]:
#Filtrar datos anomalos

In [14]:
# Nan para los datos faltantes es mejor que dropear todo el dato 
# Los nros estan masomenos pensados MASOMENOS JEJE
data.loc[(data['surface_total_in_m2']>5000) | (data['surface_total_in_m2'] < 30),'surface_total_in_m2'] = np.nan
data.loc[(data['surface_covered_in_m2']>5000),'surface_covered_in_m2'] = np.nan
data.loc[data['floor']>30,'floor'] = np.nan
data.loc[data['rooms']==0,'rooms'] = np.nan
data.loc[data['rooms']>10,'rooms'] = np.nan
data.loc[(data['lat']>-33) | (data['lat']<-36),['lat','lon','lat-lon']] = np.nan
data.loc[(data['lon']>-57) | (data['lon']<-60),['lat','lon','lat-lon']] = np.nan
data.loc[(data['expenses']>14000) | (data['expenses']<100),'expenses'] = np.nan

In [15]:
data.describe()

,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,month,year,day
count,2.241940e+05,201991.000000,201991.000000,2.847320e+05,2.847320e+05,2.847320e+05,246119.000000,257159.000000,2.012090e+05,2.541550e+05,35771.000000,168593.000000,33221.000000,284732.000000,284732.000000,284732.000000
mean,3.498000e+06,-34.580672,-58.504874,3.346441e+05,4.267056e+06,2.545583e+05,203.475530,125.382332,2.001651e+03,4.427643e+03,4.437310,2.920525,2868.053972,6.425628,2016.229497,17.467334
std,4.424446e+05,0.111066,0.151545,7.408342e+05,6.123039e+06,3.653688e+05,319.993164,159.775597,4.586174e+03,2.073286e+04,3.848211,1.409492,2580.415831,2.818749,0.903856,8.611257
min,3.427208e+06,-35.765171,-59.664010,2.600000e+03,4.757708e+04,2.754180e+03,30.000000,0.000000,8.000000e-02,5.000000e-05,1.000000,1.000000,100.000000,1.000000,2012.000000,1.000000
25%,3.429595e+06,-34.628929,-58.579640,1.050000e+05,1.660724e+06,9.900000e+04,54.000000,46.000000,1.120690e+03,1.527778e+03,2.000000,2.000000,1100.000000,4.000000,2016.000000,10.000000
50%,3.430787e+06,-34.592502,-58.480858,1.720000e+05,2.683515e+06,1.600000e+05,93.000000,77.000000,1.785714e+03,2.095532e+03,3.000000,3.000000,2000.000000,7.000000,2017.000000,19.000000
75%,3.435719e+06,-34.518706,-58.417173,3.270000e+05,4.808730e+06,2.890000e+05,215.000000,160.000000,2.450000e+03,2.896774e+03,6.000000,4.000000,3900.000000,8.000000,2017.000000,24.000000
max,6.693230e+06,-33.677756,-57.848496,1.234568e+08,7.984250e+08,5.000000e+07,5000.000000,5000.000000,1.271027e+06,4.000000e+06,30.000000,10.000000,14000.000000,12.000000,2017.000000,31.000000


In [16]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284732 entries, 0 to 317037
Data columns (total 26 columns):
id                            284732 non-null object
created_on                    284732 non-null datetime64[ns]
property_type                 284732 non-null object
place_name                    278472 non-null object
place_with_parent_names       284732 non-null object
state_name                    284732 non-null object
geonames_id                   224194 non-null float64
lat-lon                       201991 non-null object
lat                           201991 non-null float64
lon                           201991 non-null float64
price                         284732 non-null float64
currency                      284732 non-null object
price_aprox_local_currency    284732 non-null float64
price_aprox_usd               284732 non-null float64
surface_total_in_m2           246119 non-null float64
surface_covered_in_m2         257159 non-null float64
price_usd_per_m2         

In [17]:
# Dropeo las columnas que no se usan porque el test no las tiene o no son necesarias
data.drop(['price','price_usd_per_m2','price_per_m2','currency',\
          'id','created_on','geonames_id','lat-lon','price_aprox_local_currency',\
         'title','place_with_parent_names'], axis=1, inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284732 entries, 0 to 317037
Data columns (total 15 columns):
property_type            284732 non-null object
place_name               278472 non-null object
state_name               284732 non-null object
lat                      201991 non-null float64
lon                      201991 non-null float64
price_aprox_usd          284732 non-null float64
surface_total_in_m2      246119 non-null float64
surface_covered_in_m2    257159 non-null float64
floor                    35771 non-null float64
rooms                    168593 non-null float64
expenses                 33221 non-null float64
description              284718 non-null object
month                    284732 non-null int64
year                     284732 non-null int64
day                      284732 non-null int64
dtypes: float64(8), int64(3), object(4)
memory usage: 44.8+ MB


In [18]:

#test.drop(['id','created_on','lat-lon','description','country_name'], axis=1, inplace=True)
data.dropna(thresh=9,inplace=True)

In [19]:
#Esto es importante, convierto los datos categoricos en numericos cada categoria una columna con 1 si true o 0 si false
##https://stackoverflow.com/questions/34265102/xgboost-categorical-variables-dummification-vs-encoding
##https://machinelearningmastery.com/data-preparation-gradient-boosting-xgboost-python/
data = data.join(pd.get_dummies(data.property_type))
data.drop('property_type',axis=1, inplace=True)
data = data.join(data.description.str.get_dummies())
data.drop('description',axis=1, inplace=True)
data = data.join(pd.get_dummies(data.place_name))
data.drop('place_name',axis=1, inplace=True)
data = data.join(pd.get_dummies(data.state_name))
data.drop('state_name',axis=1, inplace=True)

In [20]:
#####XGBOOST
#https://github.com/dmlc/xgboost
#https://xgboost.readthedocs.io/en/latest/build.html
#https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
#https://www.slideshare.net/ShangxuanZhang/kaggle-winning-solution-xgboost-algorithm-let-us-learn-from-its-author


In [21]:
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from xgboost import XGBRegressor
import scipy.stats as st

from sklearn.metrics import mean_squared_error

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [22]:
#divide el set de datos en parte de entrenamiento y parte de test
# no se puede usar el test bajado de kaggle porque no tiene el precio
X = data.drop(["price_aprox_usd"], axis=1)
Y = data["price_aprox_usd"]

In [24]:
## Random search de hiperparametros y entrenamiento del algoritmo, 
## cuando termina tira dos scores (no se cual es mejor) y los mejores hiperparametros que dieron esos scores
## ES LA PARTE QUE TARDA
#http://danielhnyk.cz/how-to-use-xgboost-in-python/
one_to_left = st.beta(10, 1)  
from_zero_positive = st.expon(0, 50)

params = {  
    "n_estimators": [200],#nros random desde a hasta b
    "max_depth": st.randint(20, 30),
    "learning_rate": [0.0547], # nros random de una distribucion uniforme entre a y a+b
    "colsample_bytree": st.uniform(0.5,0.3), 
    "subsample": st.uniform(0.5,0.3), 
    "gamma": st.uniform(8.5, 1),
    'reg_alpha': [20,25,30,35,40],
    "min_child_weight": [15,18,20,22,25]
}

xgb = XGBRegressor(nthread=2) 

grid = RandomizedSearchCV(xgb, params, n_jobs=1,n_iter=5,verbose=10,cv=5) # n_iters es la cant de veces que busca, 10 es lo default
grid.fit(X, Y, eval_metric="rmse")

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28 
[CV]  learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28, score=0.45398050614005747, total=16.3min
[CV] learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 17.1min remaining:    0.0s


[CV]  learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28, score=0.6833677656209565, total=16.0min
[CV] learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 33.9min remaining:    0.0s


[CV]  learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28, score=0.49123252663285844, total=15.9min
[CV] learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28 


...

[CV]  learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28, score=0.6376122792316379, total=16.0min
[CV] learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28 


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed: 67.2min remaining:    0.0s


[CV]  learning_rate=0.0547, subsample=0.647635921028, colsample_bytree=0.773814799143, n_estimators=200, gamma=9.00905896034, min_child_weight=20, reg_alpha=25, max_depth=28, score=0.6361981430385704, total=16.2min
[CV] learning_rate=0.0547, subsample=0.532332901635, colsample_bytree=0.727280428048, n_estimators=200, gamma=9.21215067705, min_child_weight=22, reg_alpha=30, max_depth=27 


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 84.2min remaining:    0.0s


KeyboardInterrupt: 

In [ ]:
print(grid.best_score_)
#print(grid.best_estimator_)
print(grid.best_params_)
#print(grid.cv_results_)

In [24]:
# Parte para probar los parametros sin random search
# no tarda mucho porque tira el score de la combinacion de parametros que le pones nada mas
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,test_size=0.25)
xgb = XGBRegressor(colsample_bytree=0.98,gamma=9.33,learning_rate=0.055,max_depth=22,min_child_weight=20,n_estimators=200,\
                  reg_alpha=30,subsample=0.96,nthread=2) 

xgb.fit(X_train, Y_train,eval_metric="rmse")


XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.98,
       gamma=9.33, learning_rate=0.055, max_delta_step=0, max_depth=22,
       min_child_weight=20, missing=None, n_estimators=200, nthread=2,
       objective='reg:linear', reg_alpha=30, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.96)

In [25]:
# Print the rmse score
print(mean_squared_error(Y_test, xgb.predict(X_test)))

24415824749.1


In [83]:
#Prediccion

In [159]:
test = pd.read_csv('data/properati_dataset_testing_noprice.csv')

In [160]:
#level_0, ph, Buenos Aires Interior, index, casa, departamento
test.loc[test.state_name.str.contains('Interior'),'place_name'] = 'Palermo'
test.loc[test.state_name.str.contains('Interior'),'state_name'] = 'Capital Federal'
test.loc[test.property_type.str.contains('departamento'),'property_type'] = 'apartment'
test.loc[test.property_type.str.contains('casa'),'property_type'] = 'house'
test.loc[test.property_type.str.contains('ph'),'property_type'] = 'PH'


In [161]:
def convert(s):
    try:
        return float(s)
    except:
        return np.nan

In [162]:
test.loc[test.description.notnull(),'description'] = test.loc[test.description.notnull(),'description'].apply(descripcion_token)
test.loc[:,'place_name'] = test.place_with_parent_names.apply(place_name_from_parent)
test.loc[:,'place_name'] = test.place_name.apply(rescatar_barrios_no_oficiales)
test.loc[test.surface_total_in_m2.isnull(),'surface_total_in_m2'] = test.surface_covered_in_m2
test.loc[test.rooms.isnull(),'rooms'] = test.description.apply(recuperar_cant_amb)
test['created_on'] = pd.to_datetime(test['created_on']) ## conversion de tipo
test['month'] = test.created_on.apply(lambda x: x.month)
test['year'] = test.created_on.apply(lambda x: x.year)
test['day'] = test.created_on.apply(lambda x: x.day)
test.drop(['id','created_on','operation','lat-lon','place_with_parent_names','country_name'], axis=1, inplace=True)
test.expenses = test.expenses.apply(convert)

In [163]:
test.describe()

,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,month,year,day
count,10487.000000,10487.000000,1.339200e+04,1.300500e+04,1368.000000,7500.000000,2.523000e+03,14166.0,14166.0,14166.000000
mean,-34.629923,-58.465820,3.934301e+02,3.594716e+02,10.679094,2.804667,8.522019e+03,8.0,2017.0,14.021530
std,0.101952,0.164709,2.805803e+04,2.846901e+04,93.466971,1.425710,1.916011e+05,0.0,0.0,8.030705
min,-38.416097,-63.616672,0.000000e+00,1.000000e+00,1.000000,1.000000,0.000000e+00,8.0,2017.0,1.000000
25%,-34.643998,-58.499255,4.600000e+01,4.200000e+01,2.000000,2.000000,1.000000e+03,8.0,2017.0,9.000000
50%,-34.612651,-58.443276,7.200000e+01,6.400000e+01,4.000000,3.000000,2.000000e+03,8.0,2017.0,14.000000
75%,-34.589405,-58.401978,1.450000e+02,1.120000e+02,7.000000,4.000000,3.600000e+03,8.0,2017.0,20.000000
max,-34.536105,-57.900605,3.243242e+06,3.243242e+06,2509.000000,21.000000,7.600072e+06,8.0,2017.0,29.000000


In [164]:
# Nan para los datos faltantes es mejor que dropear todo el dato 
# Los nros estan masomenos pensados MASOMENOS JEJE
test.loc[(test['surface_total_in_m2']>5000) | (test['surface_total_in_m2'] < 30),'surface_total_in_m2'] = np.nan
test.loc[(test['surface_covered_in_m2']>5000),'surface_covered_in_m2'] = np.nan
test.loc[test['floor']>30,'floor'] = np.nan
test.loc[test['rooms']==0,'rooms'] = np.nan
test.loc[test['rooms']>10,'rooms'] = np.nan
test.loc[(test['lat']>-33) | (test['lat']<-36),['lat','lon']] = np.nan
test.loc[(test['lon']>-57) | (test['lon']<-60),['lat','lon']] = np.nan
test.loc[(test['expenses']>14000) | (test['expenses']<100),'expenses'] = np.nan

In [165]:
test.describe()

,lat,lon,surface_total_in_m2,surface_covered_in_m2,floor,rooms,expenses,month,year,day
count,10483.000000,10483.000000,12817.000000,12998.000000,1354.000000,7490.000000,2279.000000,14166.0,14166.0,14166.000000
mean,-34.628479,-58.463854,134.013654,95.626943,5.118907,2.788919,2882.024133,8.0,2017.0,14.021530
std,0.070184,0.130426,187.645059,126.166300,4.215539,1.355359,2476.270406,0.0,0.0,8.030705
min,-35.123100,-58.975341,30.000000,1.000000,1.000000,1.000000,120.000000,8.0,2017.0,1.000000
25%,-34.643941,-58.499089,48.000000,42.000000,2.000000,2.000000,1200.000000,8.0,2017.0,9.000000
50%,-34.612629,-58.443256,75.000000,64.000000,4.000000,3.000000,2000.000000,8.0,2017.0,14.000000
75%,-34.589392,-58.401970,150.000000,111.000000,7.000000,4.000000,3621.000000,8.0,2017.0,20.000000
max,-34.536105,-57.900605,4810.000000,4733.000000,29.000000,10.000000,14000.000000,8.0,2017.0,29.000000


In [172]:
test.join(test.description.str.get_dummies()).columns.str.contains('level')

AttributeError: 'numpy.ndarray' object has no attribute 'count'

In [173]:
test = test.join(pd.get_dummies(test.property_type))
test.drop('property_type',axis=1, inplace=True)
test = test.join(test.description.str.get_dummies())
test.drop('description',axis=1, inplace=True)
test = test.join(pd.get_dummies(test.place_name))
test.drop('place_name',axis=1, inplace=True)
test = test.join(pd.get_dummies(test.state_name))
test.drop('state_name',axis=1, inplace=True)

In [179]:
tot = pd.concat([X,test])
tot.reset_index(inplace=True,drop=True)

In [180]:
tot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 295475 entries, 0 to 295474
Columns: 136 entries, Abasto to year
dtypes: float64(20), int64(32), uint8(84)
memory usage: 140.9 MB


In [185]:
test_2 = tot[281309:]
test_2.reset_index(inplace=True,drop=True)

In [187]:
test_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 281309 entries, 0 to 317037
Columns: 136 entries, lat to Capital Federal
dtypes: float64(7), int64(32), uint8(97)
memory usage: 121.9 MB


In [189]:
test_2 = test_2[X.columns]

In [190]:
test_2.surface_covered_in_m2.describe()

count    12998.000000
mean        95.626943
std        126.166300
min          1.000000
25%         42.000000
50%         64.000000
75%        111.000000
max       4733.000000
Name: surface_covered_in_m2, dtype: float64

In [92]:
xgb = XGBRegressor(colsample_bytree=0.98,gamma=9.33,learning_rate=0.055,max_depth=22,min_child_weight=20,n_estimators=200,\
                  reg_alpha=30,subsample=0.96,nthread=2) 

xgb.fit(X, Y,eval_metric="rmse")


XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.98,
       gamma=9.33, learning_rate=0.055, max_delta_step=0, max_depth=22,
       min_child_weight=20, missing=None, n_estimators=200, nthread=2,
       objective='reg:linear', reg_alpha=30, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=0.96)

In [198]:
pred = xgb.predict(test_2)

In [203]:
test_e = pd.read_csv('data/properati_dataset_testing_noprice.csv')
test_e = test_e.loc[:,['id','description']]

In [204]:
test_e['price_usd'] = pred

In [206]:
test_e.drop('description',axis=1,inplace=True)

In [208]:
test_e.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14166 entries, 0 to 14165
Data columns (total 2 columns):
id           14166 non-null int64
price_usd    14166 non-null float32
dtypes: float32(1), int64(1)
memory usage: 166.1 KB


In [209]:
test_e.to_csv('Predicciones/XgBoost.csv', index = False)